In [1]:
import os
import tensorflow as tf
from tensorboard.plugins import projector

# Load model

In [ ]:
# $ tensorboard --logdir ./word_embeddings/{model_name}

In [39]:
from transformers import AutoTokenizer, TFAutoModel

In [41]:
model_names = ["bert-base-uncased", "bert-base-cased", "bert-large-uncased",
               "bert-large-cased", "bert-base-multilingual-uncased", "bert-base-multilingual-cased",
               "bert-base-chinese", "bert-base-german-cased", "bert-large-uncased-whole-word-masking",
               "bert-large-cased-whole-word-masking", "bert-large-uncased-whole-word-masking-finetuned-squad",
               "bert-large-cased-whole-word-masking-finetuned-squad", "bert-base-cased-finetuned-mrpc",
               "bert-base-german-dbmdz-cased", "bert-base-german-dbmdz-uncased"]

for model_name in model_names:
    tokenizer = AutoTokenizer.from_pretrained(
        model_name, return_tensors="tf")
    model = TFAutoModel.from_pretrained(model_name)

    log_dir = './word_embeddings/{}/'.format(model_name)

    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
        for i in range(tokenizer.vocab_size):
            f.write("{}\n".format(tokenizer.convert_ids_to_tokens(i)))

    weights = tf.Variable(model.bert.embeddings.weights[0])
    checkpoint = tf.train.Checkpoint(embedding=weights)
    checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
    embedding.metadata_path = 'metadata.tsv'
    projector.visualize_embeddings(log_dir, config)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are init

In [36]:
model.bert.embeddings.weights[0].shape

TensorShape([28996, 768])

In [10]:
#Position Embedding 
# weights = tf.Variable(model.bert.embeddings.weights[1])
# checkpoint = tf.train.Checkpoint(embedding=weights)
# checkpoint.save(os.path.join(log_dir, "position_embedding.ckpt"))

# config = projector.ProjectorConfig()
# embedding = config.embeddings.add()
# embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
# embedding.metadata_path = 'metadata.tsv'
# projector.visualize_embeddings(log_dir, config)


'./bert/position_embedding.ckpt-1'